<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/feature%2Fhybrid-models/models/hybrid_models_enconders_layering_w3_ST-HybridWaveStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta-Modelo Convolucional: ElevClusConvPrecipMetaNet

Este modelo implementa una arquitectura convolucional avanzada para la predicción espaciotemporal de precipitación con horizonte de 12 meses.

## Arquitectura

1. **Entradas**:
   - Mapas de predicción de los modelos base (ConvBiGRU-AE y ConvLSTM-AE) para cada horizonte (12 meses)
   - Información de elevación y clusters para condicionamiento (FiLM)

2. **Reducción Temprana de Canales**:
   - Reduce los 24 canales (2 modelos × 12 horizontes) a 16 para optimizar memoria
   - Aplica Conv2D(1×1) para mezclar información sin perder resolución espacial

3. **Bloques Residuales Multiescala**:
   - Bloques depthwise-separables con distintas dilataciones (1,2,4)
   - Captura patrones a diferentes escalas espaciales sin incrementar parámetros

4. **Atención Espacial por Cluster**:
   - FiLM (Feature-wise Linear Modulation): γ_cluster ⊗ F + β_cluster
   - Adapta el comportamiento según el régimen orográfico

5. **U-Net Compacto**:
   - Arquitectura de encoder-decoder con skip connections
   - Solo 2 niveles de downsampling para preservar detalle

6. **Agrupamiento de Horizontes**:
   - Conv3D para procesar conjuntamente la dimensión temporal de horizontes
   - Permite aprender relaciones entre meses consecutivos

7. **Salida Multi-Horizonte**:
   - Genera los 12 mapas refinados de predicción

8. **Estrategias Memory-Friendly**:
   - Mixed precision (float16)
   - Gradient checkpointing
   - Acumulación de gradientes
   - Entrenamiento por etapas

In [ ]:
# Predicción Espaciotemporal de Precipitación Mensual - Notebook Completo

# 0) Configuración del entorno, rutas y dependencias
import sys
import os
import logging
from pathlib import Path
import joblib  # Para persistir scalers
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

def log_and_print(msg):
    logger.info(msg)
    print(msg)

# Detectar entorno (Colab o local)
IN_COLAB = "google.colab" in sys.modules
log_and_print(f"Ejecutando en Colab: {IN_COLAB}")

# Definir rutas base
desired_repo = 'ml_precipitation_prediction'
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_PATH = Path('/content/drive/MyDrive') / desired_repo
    if not Path(desired_repo).exists():
        log_and_print("Clonando repositorio...")
        get_ipython().system('git clone https://github.com/ninja-marduk/ml_precipitation_prediction.git')
    os.chdir(desired_repo)
    get_ipython().system('pip install -q xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn ace_tools_open cartopy geopandas joblib')
else:
    # Instalación más simple sin PyEMD
    %pip install -q xarray netCDF4 scikit-image
    current = Path.cwd()
    for p in [current] + list(current.parents):
        if (p / '.git').is_dir() or (p / 'requirements.txt').is_file() or (p / 'README.md').is_file():
            BASE_PATH = p
            break
    else:
        BASE_PATH = current
    log_and_print(f"Ejecutando en local. Base path: {BASE_PATH}")

# Rutas de datos y modelos
DATA_OUTPUT   = BASE_PATH / 'data' / 'output'
MODELS_OUTPUT = BASE_PATH / 'models' / 'output'
PREDS_DIR     = MODELS_OUTPUT / 'base_model_predictions'
SHP_PATH      = BASE_PATH / 'data' / 'input' / 'shapes' / 'MGN_Departamento.shp'

# Crear directorios
MODELS_OUTPUT.mkdir(parents=True, exist_ok=True)
PREDS_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros generales
INPUT_WINDOW   = 60
OUTPUT_HORIZON = 12  # 12 meses
BATCH_SIZE     = 16
MAX_EPOCHS     = 300
PATIENCE       = 50
LR             = 1e-3

import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log_and_print(f"Usando dispositivo: {DEVICE}")


# 1) Imports adicionales y utilidades
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature  # Añadido para características cartográficas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler # Mixed precision training
from torch.utils.checkpoint import checkpoint # Gradient checkpointing
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Añadido para learning rate adaptativo
from IPython.display import clear_output  # Para actualizar gráficos durante entrenamiento
import pywt
from scipy.signal import hilbert
from skimage.restoration import denoise_wavelet

# 2) Carga y preprocesamiento de datos - Versión simplificada

def load_and_preprocess_data():
    log_and_print("Cargando datos...")
    
    # Cargar datos completos
    ds_full = xr.open_dataset(DATA_OUTPUT / 'complete_dataset_with_features_with_clusters_elevation_with_windows.nc')
    log_and_print(f"Dataset completo cargado, dims: {ds_full.dims}")
    
    # Cargar componentes directamente de archivos específicos
    ds_ceemdan = xr.open_dataset(MODELS_OUTPUT / 'features_CEEMDAN.nc')
    log_and_print(f"Dataset CEEMDAN cargado, dims: {ds_ceemdan.dims}")
    log_and_print(f"Variables disponibles en CEEMDAN: {list(ds_ceemdan.data_vars.keys())}")
    
    ds_tvfemd = xr.open_dataset(MODELS_OUTPUT / 'features_TVFEMD.nc')
    log_and_print(f"Dataset TVF-EMD cargado, dims: {ds_tvfemd.dims}")
    log_and_print(f"Variables disponibles en TVF-EMD: {list(ds_tvfemd.data_vars.keys())}")
    
    # Cargar shapefile para visualizaciones
    gdf = gpd.read_file(SHP_PATH)
    if gdf.crs is None:
        gdf = gdf.set_crs(epsg=4326)
    elif gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    log_and_print("Shapefile cargado y CRS validado.")

    # Extraer información temporal
    times = ds_full.time.values.astype('datetime64[M]')
    REF = np.datetime64('2024-02','M')
    idx_ref = int(np.where(times==REF)[0][0])
    log_and_print(f"Referencia (REF) = {REF}, index={idx_ref}")
    
    return ds_full, ds_ceemdan, ds_tvfemd, gdf, times, REF, idx_ref

# Ejecutar carga/preproc con los archivos específicos
ds_full, ds_ceemdan, ds_tvfemd, gdf, times, REF, idx_ref = load_and_preprocess_data()

# Mantenemos sólo las funciones de preprocesamiento que aún se necesitan
def calculate_afc(signal, lags=[1, 3, 6]):
    """Calcula la Función de Autocorrelación para los lags dados."""
    afc = [np.correlate(signal[lag:], signal[:-lag], mode='valid') for lag in lags]
    return afc

# Funciones de preprocesamiento adicionales
def wavelet_denoise(data, wavelet='db4', level=3):
    """Aplica denoising wavelet a los datos."""
    try:
        # API moderna de scikit-image (0.19+)
        return denoise_wavelet(
            data, 
            wavelet=wavelet, 
            mode='soft',
            method='BayesShrink',
            channel_axis=None
        )
    except Exception as e:
        print(f"Error en wavelet denoising: {str(e)}. Intentando método alternativo.")
        # Si falla, intentar solo con parámetros básicos
        return denoise_wavelet(data, wavelet=wavelet)

# 3) Definición de Datasets PyTorch
class PrecipitationDataset(Dataset):
    """
    Dataset personalizado para datos de precipitación que maneja la reducción
    de dimensionalidad y garantiza la compatibilidad dimensional.
    """
    def __init__(self, data, target, seq_length):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).float()
        self.seq_length = seq_length
        self.target_shape = target.shape[1:]  # Guardar la forma objetivo (height, width)
        
        print(f"Dataset inicializado - Forma de datos: {self.data.shape}")
        print(f"Dataset inicializado - Forma de targets: {self.target.shape}")
        print(f"Forma objetivo almacenada: {self.target_shape}")

    def __len__(self):
        return len(self.data) - self.seq_length + 1

    def __getitem__(self, idx):
        # Obtener secuencia de datos
        inputs = self.data[idx:idx+self.seq_length]
        labels = self.target[idx:idx+self.seq_length]
        
        # Para debugging, imprimir formas solo para el primer elemento
        if idx == 0:
            print(f"Ejemplo de entrada - forma original: {inputs.shape}")
            
        # Simplificar dimensiones si es posible
        if len(inputs.shape) > 3 and inputs.shape[1] == 1:  # Si hay una sola ventana/característica
            inputs = inputs.squeeze(1)  # Eliminar dimensión redundante 
            
        # Para debugging
        if idx == 0:
            print(f"Ejemplo de entrada - forma final: {inputs.shape}")
            print(f"Ejemplo de etiqueta: {labels.shape}")
            
        return inputs, labels

# 4) Modelos Híbridos: ConvBiGRU-AE y ConvLSTM-AE
class ConvBiGRU_AE(nn.Module):
    def __init__(self, input_channels, hidden_dim, num_layers, output_channels, seq_length, target_shape=(61, 65), kernel_size=3, padding=1):
        super(ConvBiGRU_AE, self).__init__()
        self.seq_length = seq_length
        self.hidden_dim = hidden_dim
        self.output_channels = output_channels
        self.target_shape = target_shape  # Guardar la forma objetivo
        
        # Encoder - Convoluciones 2D para cada paso de tiempo
        # En lugar de esperar múltiples canales, procesaremos cada paso de tiempo independientemente
        self.conv1 = nn.Conv2d(1, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm1 = nn.InstanceNorm2d(hidden_dim)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm2 = nn.InstanceNorm2d(hidden_dim)
        
        # GRU para procesar la secuencia comprimida
        self.gru = nn.GRU(hidden_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        
        # Decoder
        self.deconv1 = nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.dnorm1 = nn.InstanceNorm2d(hidden_dim)
        self.deconv2 = nn.ConvTranspose2d(hidden_dim, output_channels * seq_length, kernel_size=kernel_size, padding=padding)
        
    def forward(self, x, target_shape=None):
        # Usar la forma objetivo proporcionada o la predeterminada
        if target_shape is None:
            target_shape = self.target_shape
        
        # x tiene forma [batch_size, seq_length, height, width]
        batch_size, seq_len, H, W = x.size()
        
        # Procesar cada paso de tiempo individualmente
        processed_features = []
        
        for t in range(seq_len):
            # Obtener el paso de tiempo actual y añadir dimensión de canal
            x_t = x[:, t].unsqueeze(1)  # [batch_size, 1, height, width]
            
            # Aplicar convoluciones
            x_t = F.relu(self.norm1(self.conv1(x_t)))
            x_t = F.relu(self.norm2(self.conv2(x_t)))
            
            # Extraer características globales
            # Promedio espacial para reducir a [batch_size, hidden_dim]
            x_t_features = x_t.mean(dim=(2, 3))
            
            # Almacenar características para este paso de tiempo
            processed_features.append(x_t_features)
        
        # Concatenar características para todos los pasos de tiempo
        sequence_features = torch.stack(processed_features, dim=1)  # [batch_size, seq_length, hidden_dim]
        
        # Aplicar GRU a la secuencia
        output, _ = self.gru(sequence_features)  # [batch_size, seq_length, hidden_dim*2]
        
        # Tomar el último estado y preparar para deconv
        output = output[:, -1, :].view(batch_size, self.hidden_dim*2, 1, 1)
        output = F.interpolate(output, size=(H, W), mode='nearest')
        
        # Decoder
        output = F.relu(self.dnorm1(self.deconv1(output)))
        output = self.deconv2(output)  # [batch_size, seq_length*output_channels, H, W]
        
        # Reorganizar para obtener [batch_size, seq_length, output_channels, H, W]
        output = output.view(batch_size, self.seq_length, self.output_channels, H, W)
        
        # Redimensionar a la forma objetivo usando interpolación bilineal
        # Primero reorganizamos para [batch_size*seq_length, output_channels, H, W]
        output_reshaped = output.view(batch_size * self.seq_length, self.output_channels, H, W)
        output_resized = F.interpolate(output_reshaped, size=target_shape, mode='bilinear', align_corners=False)
        
        # Reorganizar de nuevo a [batch_size, seq_length, output_channels, target_H, target_W]
        output = output_resized.view(batch_size, self.seq_length, self.output_channels, target_shape[0], target_shape[1])
        
        return output

class ConvBiLSTM_AE(nn.Module):
    def __init__(self, input_channels, hidden_dim, num_layers, output_channels, seq_length, target_shape=(61, 65), kernel_size=3, padding=1):
        super(ConvBiLSTM_AE, self).__init__()
        self.seq_length = seq_length
        self.hidden_dim = hidden_dim
        self.output_channels = output_channels
        self.target_shape = target_shape  # Guardar la forma objetivo
        
        # Encoder para procesar cada paso de tiempo individualmente
        self.conv1 = nn.Conv2d(1, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm1 = nn.InstanceNorm2d(hidden_dim)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.norm2 = nn.InstanceNorm2d(hidden_dim)
        
        # LSTM para procesar la secuencia
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
        
        # Decoder
        self.deconv1 = nn.ConvTranspose2d(hidden_dim * 2, hidden_dim, kernel_size=kernel_size, padding=padding)
        self.dnorm1 = nn.InstanceNorm2d(hidden_dim)
        self.deconv2 = nn.ConvTranspose2d(hidden_dim, output_channels * seq_length, kernel_size=kernel_size, padding=padding)
        
    def forward(self, x, target_shape=None):
        # Usar la forma objetivo proporcionada o la predeterminada
        if target_shape is None:
            target_shape = self.target_shape
            
        # x tiene forma [batch_size, seq_length, height, width]
        batch_size, seq_len, H, W = x.size()
        
        # Procesar cada paso de tiempo individualmente
        processed_features = []
        
        for t in range(seq_len):
            # Obtener el paso de tiempo actual y añadir dimensión de canal
            x_t = x[:, t].unsqueeze(1)  # [batch_size, 1, height, width]
            
            # Aplicar convoluciones
            x_t = F.relu(self.norm1(self.conv1(x_t)))
            x_t = F.relu(self.norm2(self.conv2(x_t)))
            
            # Extraer características globales
            # Promedio espacial para reducir a [batch_size, hidden_dim]
            x_t_features = x_t.mean(dim=(2, 3))
            
            # Almacenar características para este paso de tiempo
            processed_features.append(x_t_features)
        
        # Concatenar características para todos los pasos de tiempo
        sequence_features = torch.stack(processed_features, dim=1)  # [batch_size, seq_length, hidden_dim]
        
        # Aplicar LSTM a la secuencia
        output, _ = self.lstm(sequence_features)  # [batch_size, seq_length, hidden_dim*2]
        
        # Tomar el último estado y preparar para deconv
        output = output[:, -1, :].view(batch_size, self.hidden_dim*2, 1, 1)
        output = F.interpolate(output, size=(H, W), mode='nearest')
        
        # Decoder
        output = F.relu(self.dnorm1(self.deconv1(output)))
        output = self.deconv2(output)  # [batch_size, seq_length*output_channels, H, W]
        
        # Reorganizar para obtener [batch_size, seq_length, output_channels, H, W]
        output = output.view(batch_size, self.seq_length, self.output_channels, H, W)
        
        # Redimensionar a la forma objetivo usando interpolación bilineal
        output_reshaped = output.view(batch_size * self.seq_length, self.output_channels, H, W)
        output_resized = F.interpolate(output_reshaped, size=target_shape, mode='bilinear', align_corners=False)
        output = output_resized.view(batch_size, self.seq_length, self.output_channels, target_shape[0], target_shape[1])
        
        return output

# 4.5) Modelos ConvGRU y ConvLSTM simples para comparación
class SimpleConvGRU(nn.Module):
    """
    Modelo simple ConvGRU para comparar con los modelos híbridos.
    """
    def __init__(self, input_channels, hidden_dim, output_channels, seq_length=12, target_shape=(61, 65)):
        super(SimpleConvGRU, self).__init__()
        self.input_channels = input_channels
        self.hidden_dim = hidden_dim
        self.output_channels = output_channels
        self.seq_length = seq_length
        self.target_shape = target_shape
        
        # Capas convolucionales para procesar entrada
        self.conv1 = nn.Conv2d(input_channels, hidden_dim // 2, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_dim // 2)
        self.conv2 = nn.Conv2d(hidden_dim // 2, hidden_dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        
        # Capa GRU
        self.gru = nn.GRU(
            input_size=hidden_dim * target_shape[0] * target_shape[1], 
            hidden_size=hidden_dim * 4, 
            num_layers=2, 
            batch_first=True
        )
        
        # Capas para generar salida
        self.upconv1 = nn.Conv2d(hidden_dim * 4, hidden_dim * 2, kernel_size=3, padding=1)
        self.upbn1 = nn.BatchNorm2d(hidden_dim * 2)
        self.upconv2 = nn.Conv2d(hidden_dim * 2, hidden_dim, kernel_size=3, padding=1)
        self.upbn2 = nn.BatchNorm2d(hidden_dim)
        
        # Capa final para generar múltiples horizontes de predicción
        self.out_conv = nn.Conv2d(hidden_dim, output_channels * seq_length, kernel_size=1)
        
    def forward(self, x, target_shape=None):
        """
        Args:
            x: Tensor de entrada que puede tener varias formas:
               - [batch_size, seq_len, input_window, channels, height, width]
               - [batch_size, seq_len, channels, height, width]
               - [batch_size, channels, height, width]
            target_shape: Forma objetivo para la salida
        """
        if target_shape is None:
            target_shape = self.target_shape
        
        # NUEVO: Manejar el caso de 6 dimensiones [batch, seq, window, channel, H, W]
        if len(x.shape) == 6:  
            batch_size, seq_len, window, channels, H, W = x.shape
            
            # Reorganizar a [batch, seq, channels*window, H, W] combinando window y channels
            x = x.permute(0, 1, 3, 2, 4, 5).contiguous()  # [batch, seq, channel, window, H, W]
            x = x.view(batch_size, seq_len, channels*window, H, W)  # [batch, seq, channel*window, H, W]
            
            # Procesar cada paso de tiempo independientemente
            processed_features = []
            
            for t in range(seq_len):
                # Extraer slice temporal: [batch, channels*window, H, W]
                xt = x[:, t]
                
                # Aplicar convoluciones
                xt = F.relu(self.bn1(self.conv1(xt)))
                xt = F.relu(self.bn2(self.conv2(xt)))
                
                # Aplanar para GRU: [batch, hidden_dim*H*W]
                xt = xt.view(batch_size, -1)
                processed_features.append(xt)
            
            # Stack para secuencia: [batch, seq_len, features]
            x_sequence = torch.stack(processed_features, dim=1)
            
            # Aplicar GRU
            gru_out, _ = self.gru(x_sequence)
            
            # Tomar último estado
            last_out = gru_out[:, -1]  # [batch, hidden_dim*4]
            
        # Verificar y manejar diferentes formas de entrada
        elif len(x.shape) == 5:  # [batch, seq_len, channels, height, width]
            batch_size = x.size(0)
            seq_len = x.size(1)
            
            # Procesar cada paso de tiempo independientemente
            processed_features = []
            
            for t in range(seq_len):
                # Extraer slice temporal: [batch, channels, H, W]
                xt = x[:, t]
                
                # Aplicar convoluciones
                xt = F.relu(self.bn1(self.conv1(xt)))
                xt = F.relu(self.bn2(self.conv2(xt)))
                
                # Aplanar para GRU: [batch, hidden_dim*H*W]
                xt = xt.view(batch_size, -1)
                processed_features.append(xt)
            
            # Stack para secuencia: [batch, seq_len, features]
            x_sequence = torch.stack(processed_features, dim=1)
            
            # Aplicar GRU
            gru_out, _ = self.gru(x_sequence)
            
            # Tomar último estado
            last_out = gru_out[:, -1]  # [batch, hidden_dim*4]
            
        elif len(x.shape) == 4:  # [batch, channels, height, width] (sin dimensión de secuencia)
            batch_size = x.size(0)
            
            # Aplicar convoluciones directamente
            x = F.relu(self.bn1(self.conv1(x)))
            x = F.relu(self.bn2(self.conv2(x)))
            
            # Aplanar
            x_flat = x.view(batch_size, -1)
            
            # Añadir dimensión de secuencia ficticia
            x_sequence = x_flat.unsqueeze(1)  # [batch, 1, features]
            
            # Aplicar GRU
            gru_out, _ = self.gru(x_sequence)
            
            # Tomar salida
            last_out = gru_out[:, -1]  # [batch, hidden_dim*4]
        
        else:
            raise ValueError(f"Forma de entrada no soportada: {x.shape}")
        
        # Reformar para procesamiento convolucional
        last_out = last_out.view(batch_size, self.hidden_dim * 4, 1, 1)
        last_out = F.interpolate(last_out, size=target_shape, mode='bilinear', align_corners=False)
        
        # Capas finales
        out = F.relu(self.upbn1(self.upconv1(last_out)))
        out = F.relu(self.upbn2(self.upconv2(out)))
        out = self.out_conv(out)
        
        # Reorganizar para obtener [batch, seq_len, channels, H, W]
        out = out.view(batch_size, self.seq_length, self.output_channels, *target_shape)
        
        return out
        
class SimpleConvLSTM(nn.Module):
    """
    Modelo simple ConvLSTM para comparar con los modelos híbridos.
    """
    def __init__(self, input_channels, hidden_dim, output_channels, seq_length=12, target_shape=(61, 65)):
        super(SimpleConvLSTM, self).__init__()
        self.input_channels = input_channels
        self.hidden_dim = hidden_dim
        self.output_channels = output_channels
        self.seq_length = seq_length
        self.target_shape = target_shape
        
        # Capas convolucionales para procesar entrada
        self.conv1 = nn.Conv2d(input_channels, hidden_dim // 2, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_dim // 2)
        self.conv2 = nn.Conv2d(hidden_dim // 2, hidden_dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        
        # Capa LSTM
        self.lstm = nn.LSTM(
            input_size=hidden_dim * target_shape[0] * target_shape[1], 
            hidden_size=hidden_dim * 4, 
            num_layers=2, 
            batch_first=True
        )
        
        # Capas para generar salida
        self.upconv1 = nn.Conv2d(hidden_dim * 4, hidden_dim * 2, kernel_size=3, padding=1)
        self.upbn1 = nn.BatchNorm2d(hidden_dim * 2)
        self.upconv2 = nn.Conv2d(hidden_dim * 2, hidden_dim, kernel_size=3, padding=1)
        self.upbn2 = nn.BatchNorm2d(hidden_dim)
        
        # Capa final para generar múltiples horizontes de predicción
        self.out_conv = nn.Conv2d(hidden_dim, output_channels * seq_length, kernel_size=1)
        
    def forward(self, x, target_shape=None):
        """
        Args:
            x: Tensor de entrada que puede tener varias formas:
               - [batch_size, seq_len, input_window, channels, height, width]
               - [batch_size, seq_len, channels, height, width]
               - [batch_size, channels, height, width]
            target_shape: Forma objetivo para la salida
        """
        if target_shape is None:
            target_shape = self.target_shape
            
        # NUEVO: Manejar el caso de 6 dimensiones [batch, seq, window, channel, H, W]
        if len(x.shape) == 6:  
            batch_size, seq_len, window, channels, H, W = x.shape
            
            # Reorganizar a [batch, seq, channels*window, H, W] combinando window y channels
            x = x.permute(0, 1, 3, 2, 4, 5).contiguous()  # [batch, seq, channel, window, H, W]
            x = x.view(batch_size, seq_len, channels*window, H, W)  # [batch, seq, channel*window, H, W]
            
            # Procesar cada paso de tiempo independientemente
            processed_features = []
            
            for t in range(seq_len):
                # Extraer slice temporal: [batch, channels*window, H, W]
                xt = x[:, t]
                
                # Aplicar convoluciones
                xt = F.relu(self.bn1(self.conv1(xt)))
                xt = F.relu(self.bn2(self.conv2(xt)))
                
                # Aplanar para LSTM: [batch, hidden_dim*H*W]
                xt = xt.view(batch_size, -1)
                processed_features.append(xt)
            
            # Stack para secuencia: [batch, seq_len, features]
            x_sequence = torch.stack(processed_features, dim=1)
            
            # Aplicar LSTM
            lstm_out, _ = self.lstm(x_sequence)
            
            # Tomar último estado
            last_out = lstm_out[:, -1]  # [batch, hidden_dim*4]
            
        # Verificar y manejar diferentes formas de entrada
        elif len(x.shape) == 5:  # [batch, seq_len, channels, height, width]
            batch_size = x.size(0)
            seq_len = x.size(1)
            
            # Procesar cada paso de tiempo independientemente
            processed_features = []
            
            for t in range(seq_len):
                # Extraer slice temporal: [batch, channels, H, W]
                xt = x[:, t]
                
                # Aplicar convoluciones
                xt = F.relu(self.bn1(self.conv1(xt)))
                xt = F.relu(self.bn2(self.conv2(xt)))
                
                # Aplanar para LSTM: [batch, hidden_dim*H*W]
                xt = xt.view(batch_size, -1)
                processed_features.append(xt)
            
            # Stack para secuencia: [batch, seq_len, features]
            x_sequence = torch.stack(processed_features, dim=1)
            
            # Aplicar LSTM
            lstm_out, _ = self.lstm(x_sequence)
            
            # Tomar último estado
            last_out = lstm_out[:, -1]  # [batch, hidden_dim*4]
            
        elif len(x.shape) == 4:  # [batch, channels, height, width] (sin dimensión de secuencia)
            batch_size = x.size(0)
            
            # Aplicar convoluciones directamente
            x = F.relu(self.bn1(self.conv1(x)))
            x = F.relu(self.bn2(self.conv2(x)))
            
            # Aplanar
            x_flat = x.view(batch_size, -1)
            
            # Añadir dimensión de secuencia ficticia
            x_sequence = x_flat.unsqueeze(1)  # [batch, 1, features]
            
            # Aplicar LSTM
            lstm_out, _ = self.lstm(x_sequence)
            
            # Tomar salida
            last_out = lstm_out[:, -1]  # [batch, hidden_dim*4]
        
        else:
            raise ValueError(f"Forma de entrada no soportada: {x.shape}")
        
        # Reformar para procesamiento convolucional
        last_out = last_out.view(batch_size, self.hidden_dim * 4, 1, 1)
        last_out = F.interpolate(last_out, size=target_shape, mode='bilinear', align_corners=False)
        
        # Capas finales
        out = F.relu(self.upbn1(self.upconv1(last_out)))
        out = F.relu(self.upbn2(self.upconv2(out)))
        out = self.out_conv(out)
        
        # Reorganizar para obtener [batch, seq_len, channels, H, W]
        out = out.view(batch_size, self.seq_length, self.output_channels, *target_shape)
        
        return out

# Función para entrenar y evaluar todos los modelos disponibles
def train_and_compare_models(X_train, y_train, X_val, y_val, force_retrain=False):
    """
    Entrena y compara modelos simples y híbridos usando las mismas entradas.
    
    Args:
        X_train: Datos de entrenamiento
        y_train: Etiquetas de entrenamiento
        X_val: Datos de validación
        y_val: Etiquetas de validación
        force_retrain: Si reentrenar aunque existan modelos guardados
    
    Returns:
        dict: Diccionario con modelos entrenados y sus métricas
    """
    print("\n" + "="*70)
    print("ENTRENAMIENTO Y COMPARACIÓN DE MODELOS SIMPLES Y HÍBRIDOS")
    print("="*70)
    
    # Directorios para guardar modelos
    models_dir = MODELS_OUTPUT / 'comparison_models'
    models_dir.mkdir(exist_ok=True, parents=True)
    
    # Parámetros comunes
    input_channels = X_train.shape[1] if len(X_train.shape) > 3 else 1
    hidden_dim = 128
    output_channels = 1
    seq_length = OUTPUT_HORIZON
    
    # Obtener shape de target si está disponible
    if len(y_train.shape) >= 3:
        target_shape = y_train.shape[-2:]
    else:
        target_shape = (61, 65)  # valores por defecto
    
    print(f"Configuración común: input_channels={input_channels}, hidden_dim={hidden_dim}, output_channels={output_channels}")
    print(f"Secuencia: {seq_length}, target_shape: {target_shape}")
    
    # Definir los modelos a entrenar con sus rutas de guardado
    models = {
        'SimpleConvGRU': {
            'class': SimpleConvGRU,
            'path': models_dir / 'simple_convgru.pth',
            'params': {
                'input_channels': input_channels,
                'hidden_dim': hidden_dim,
                'output_channels': output_channels,
                'seq_length': seq_length,
                'target_shape': target_shape
            },
            'metrics': {},
            'train_losses': [],
            'val_losses': []
        },
        'SimpleConvLSTM': {
            'class': SimpleConvLSTM,
            'path': models_dir / 'simple_convlstm.pth',
            'params': {
                'input_channels': input_channels,
                'hidden_dim': hidden_dim,
                'output_channels': output_channels,
                'seq_length': seq_length,
                'target_shape': target_shape
            },
            'metrics': {},
            'train_losses': [],
            'val_losses': []
        },
        'ConvBiGRU-AE': {
            'class': ConvBiGRU_AE,
            'path': models_dir / 'convbigru_ae.pth',
            'params': {
                'input_channels': input_channels,
                'hidden_dim': hidden_dim,
                'num_layers': 3,
                'output_channels': output_channels,
                'seq_length': seq_length,
                'target_shape': target_shape
            },
            'metrics': {},
            'train_losses': [],
            'val_losses': []
        },
        'ConvBiLSTM-AE': {
            'class': ConvBiLSTM_AE,
            'path': models_dir / 'convbilstm_ae.pth',
            'params': {
                'input_channels': input_channels,
                'hidden_dim': hidden_dim,
                'num_layers': 3,
                'output_channels': output_channels,
                'seq_length': seq_length,
                'target_shape': target_shape
            },
            'metrics': {},
            'train_losses': [],
            'val_losses': []
        }
    }
    
    # Crear datasets y dataloaders
    train_dataset = PrecipitationDataset(X_train, y_train, seq_length)
    val_dataset = PrecipitationDataset(X_val, y_val, seq_length)
    
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Iterar sobre cada modelo
    for model_name, config in models.items():
        print(f"\n{'-'*50}")
        print(f"PROCESANDO MODELO: {model_name}")
        print(f"{'-'*50}")
        
        # Verificar si existe modelo guardado
        if config['path'].exists() and not force_retrain:
            print(f"Modelo encontrado en {config['path']}, cargando...")
            try:
                # Cargar modelo existente
                model = config['class'](**config['params']).to(DEVICE)
                checkpoint = torch.load(config['path'], map_location=DEVICE)
                
                # Verificar contenido del checkpoint
                if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                    model.load_state_dict(checkpoint['model_state_dict'])
                    config['train_losses'] = checkpoint.get('train_losses', [])
                    config['val_losses'] = checkpoint.get('val_losses', [])
                else:
                    # Formato antiguo (solo state_dict)
                    model.load_state_dict(checkpoint)
                
                print(f"✅ Modelo {model_name} cargado correctamente")
            except Exception as e:
                print(f"❌ Error al cargar modelo: {str(e)}")
                print("Entrenando el modelo desde cero...")
                
                # Instanciar modelo
                model = config['class'](**config['params']).to(DEVICE)
                
                # Configurar entrenamiento
                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
                scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
                
                # Entrenar modelo
                model, train_losses, val_losses = train_hybrid_model(
                    name=model_name,
                    model=model,
                    train_loader=train_loader,
                    val_loader=val_loader,
                    epochs=100,
                    patience=20,
                    optimizer=optimizer,
                    criterion=criterion,
                    scheduler=scheduler
                )
                
                # Guardar historial de pérdidas
                config['train_losses'] = train_losses
                config['val_losses'] = val_losses
                
                # Guardar modelo
                save_data = {
                    'model_state_dict': model.state_dict(),
                    'train_losses': train_losses,
                    'val_losses': val_losses
                }
                torch.save(save_data, config['path'])
        else:
            # Entrenar modelo
            print(f"Entrenando {model_name} desde cero...")
            
            # Instanciar modelo
            model = config['class'](**config['params']).to(DEVICE)
            
            # Configurar entrenamiento
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
            
            # Entrenar modelo
            model, train_losses, val_losses = train_hybrid_model(
                name=model_name,
                model=model,
                train_loader=train_loader,
                val_loader=val_loader,
                epochs=100,
                patience=20,
                optimizer=optimizer,
                criterion=criterion,
                scheduler=scheduler
            )
            
            # Guardar historial de pérdidas
            config['train_losses'] = train_losses
            config['val_losses'] = val_losses
            
            # Guardar modelo
            save_data = {
                'model_state_dict': model.state_dict(),
                'train_losses': train_losses,
                'val_losses': val_losses
            }
            torch.save(save_data, config['path'])
        
        # Evaluar modelo con métricas adicionales
        print(f"Evaluando {model_name} con métricas adicionales...")
        model.eval()
        
        all_targets = []
        all_preds = []
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(DEVICE)
                outputs = model(inputs)
                
                # Asegurar dimensión de canal si es necesario
                if len(outputs.shape) == 5 and outputs.shape[2] == 1:
                    outputs = outputs.squeeze(2)
                
                # Transformar a numpy para métricas
                preds = outputs.cpu().numpy()
                targets_np = targets.numpy()
                
                # Almacenar para métricas globales
                all_targets.append(targets_np)
                all_preds.append(preds)
        
        # Concatenar todas las predicciones y targets
        all_targets = np.concatenate(all_targets, axis=0)
        all_preds = np.concatenate(all_preds, axis=0)
        
        # Calcular métricas
        # Aplanar para métricas generales
        flat_targets = all_targets.flatten()
        flat_preds = all_preds.flatten()
        
        # Eliminar valores NaN si existen
        mask = ~np.isnan(flat_targets) & ~np.isnan(flat_preds)
        flat_targets = flat_targets[mask]
        flat_preds = flat_preds[mask]
        
        # Calcular métricas
        mae = mean_absolute_error(flat_targets, flat_preds)
        rmse = np.sqrt(mean_squared_error(flat_targets, flat_preds))
        r2 = r2_score(flat_targets, flat_preds)
        corr = np.corrcoef(flat_targets, flat_preds)[0, 1]
        
        # Calcular MAPE evitando divisiones por cero
        mask_nonzero = flat_targets != 0
        mape = np.mean(np.abs((flat_targets[mask_nonzero] - flat_preds[mask_nonzero]) / flat_targets[mask_nonzero])) * 100
        
        # Almacenar métricas
        config['metrics'] = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE (%)': mape,
            'r': corr,
            'R²': r2
        }
        
        # Almacenar modelo en el diccionario
        config['model'] = model
        
        # Mostrar métricas
        print(f"\nMétricas para {model_name}:")
        print(f"  MAE: {mae:.4f}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAPE: {mape:.2f}%")
        print(f"  r (correlación): {corr:.4f}")
        print(f"  R²: {r2:.4f}")
    
    # Devolver diccionario con todos los modelos y resultados
    return models

# Función para generar visualizaciones comparativas
def visualize_model_comparisons(models_dict):
    """
    Genera visualizaciones comparativas para todos los modelos
    
    Args:
        models_dict: Diccionario con modelos y sus métricas
    """
    print("\n" + "="*70)
    print("GENERANDO VISUALIZACIONES COMPARATIVAS")
    print("="*70)
    
    # Crear directorio para visualizaciones
    vis_dir = MODELS_OUTPUT / 'visualization' / 'comparisons'
    vis_dir.mkdir(exist_ok=True, parents=True)
    
    # 1. Tabla comparativa de métricas
    model_names = list(models_dict.keys())
    metrics = ['MAE', 'RMSE', 'MAPE (%)', 'r', 'R²']
    
    # Crear DataFrame para tabla
    metrics_data = {metric: [] for metric in metrics}
    metrics_data['Modelo'] = model_names
    
    for model_name in model_names:
        for metric in metrics:
            value = models_dict[model_name]['metrics'].get(metric, np.nan)
            metrics_data[metric].append(value)
    
    metrics_df = pd.DataFrame(metrics_data)
    
    # Mostrar tabla
    print("\n📊 TABLA COMPARATIVA DE MÉTRICAS")
    print(metrics_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
    
    # Guardar como CSV
    metrics_csv_path = vis_dir / 'metrics_comparison.csv'
    metrics_df.to_csv(metrics_csv_path, index=False, float_format='%.4f')
    print(f"\nTabla guardada en {metrics_csv_path}")
    
    # 2. Gráfico de barras comparativo de métricas
    plt.figure(figsize=(15, 8))
    
    # Iterar sobre las métricas
    for i, metric in enumerate(metrics):
        plt.subplot(2, 3, i+1)
        values = [models_dict[model]['metrics'].get(metric, np.nan) for model in model_names]
        
        # Crear gráfico de barras
        bars = plt.bar(model_names, values)
        
        # Añadir etiquetas de valor sobre cada barra
        for bar, value in zip(bars, values):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{value:.4f}',
                    ha='center', va='bottom', rotation=0, fontsize=9)
        
        plt.title(f'Comparación de {metric}')
        plt.xticks(rotation=45, ha='right')
        plt.grid(alpha=0.3, axis='y')
        
        # Para métricas donde menor es mejor, destacar el mejor modelo
        if metric in ['MAE', 'RMSE', 'MAPE (%)']:
            best_idx = np.nanargmin(values)
            bars[best_idx].set_color('green')
        else:  # Para métricas donde mayor es mejor
            best_idx = np.nanargmax(values)
            bars[best_idx].set_color('green')
    
    plt.tight_layout()
    metrics_plot_path = vis_dir / 'metrics_comparison.png'
    plt.savefig(metrics_plot_path, dpi=300, bbox_inches='tight')
    print(f"Gráfico de métricas guardado en {metrics_plot_path}")
    plt.show()
    
    # 3. Curvas de aprendizaje comparativas
    plt.figure(figsize=(15, 8))
    
    # Pérdidas de entrenamiento
    plt.subplot(1, 2, 1)
    for model_name in model_names:
        train_losses = models_dict[model_name].get('train_losses', [])
        if train_losses:
            plt.plot(train_losses, label=model_name)
    
    plt.title('Pérdidas de Entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.grid(alpha=0.3)
    plt.legend()
    
    # Pérdidas de validación
    plt.subplot(1, 2, 2)
    for model_name in model_names:
        val_losses = models_dict[model_name].get('val_losses', [])
        if val_losses:
            plt.plot(val_losses, label=model_name)
    
    plt.title('Pérdidas de Validación')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.grid(alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    learning_curves_path = vis_dir / 'learning_curves_comparison.png'
    plt.savefig(learning_curves_path, dpi=300, bbox_inches='tight')
    print(f"Curvas de aprendizaje guardadas en {learning_curves_path}")
    plt.show()
    
    # 4. Mapa de predicciones vs reales y errores para un mes específico
    # Generar predicciones
    try:
        # Cargar datos de validación
        val_batch = next(iter(val_loader))
        inputs, targets = val_batch
        inputs = inputs.to(DEVICE)
        
        # Seleccionar un mes para visualización (por ejemplo, el tercero)
        month_idx = min(2, OUTPUT_HORIZON - 1)  # mes 3 (índice 2) o el último si el horizonte es menor
        
        plt.figure(figsize=(20, 4 * len(model_names)))
        
        # Para cada modelo
        for i, (model_name, config) in enumerate(models_dict.items()):
            model = config['model']
            model.eval()
            
            with torch.no_grad():
                outputs = model(inputs)
                
                # Ajustar dimensiones
                if len(outputs.shape) == 5 and outputs.shape[2] == 1:
                    outputs = outputs.squeeze(2)
                
                # Seleccionar primer batch y mes específico
                pred = outputs[0, month_idx].cpu().numpy()
                
                # Extraer target correspondiente
                if len(targets.shape) == 4 and targets.shape[1] > month_idx:
                    target = targets[0, month_idx].numpy()
                elif len(targets.shape) == 3:
                    target = targets[0].numpy()
                else:
                    target = targets[0, 0].numpy()
                    
                # Extraer coordenadas para visualización
                latitudes = ds_full.latitude.values
                longitudes = ds_full.longitude.values
                lon_mesh, lat_mesh = np.meshgrid(longitudes, latitudes)
                
                # Calcular error y métricas
                error = pred - target
                month_mae = np.mean(np.abs(error))
                month_rmse = np.sqrt(np.mean(error**2))
                mask_nonzero = target != 0
                month_mape = np.mean(np.abs(error[mask_nonzero] / target[mask_nonzero])) * 100 if np.any(mask_nonzero) else np.nan
                
                # Visualizar
                plt.subplot(len(model_names), 3, i*3 + 1)
                plt.pcolormesh(lon_mesh, lat_mesh, target)
                plt.colorbar(label='Precipitación (mm)')
                plt.title(f'Real - Mes {month_idx+1}')
                
                plt.subplot(len(model_names), 3, i*3 + 2)
                plt.pcolormesh(lon_mesh, lat_mesh, pred)
                plt.colorbar(label='Precipitación (mm)')
                plt.title(f'{model_name} - Predicción')
                
                plt.subplot(len(model_names), 3, i*3 + 3)
                plt.pcolormesh(lon_mesh, lat_mesh, error, cmap='RdBu_r')
                plt.colorbar(label='Error (mm)')
                plt.title(f'Error - MAE:{month_mae:.2f}, MAPE:{month_mape:.2f}%')
        
        plt.tight_layout()
        predictions_path = vis_dir / 'predictions_comparison.png'
        plt.savefig(predictions_path, dpi=300, bbox_inches='tight')
        print(f"Comparación de predicciones guardada en {predictions_path}")
        plt.show()
        
    except Exception as e:
        print(f"Error al generar visualización de predicciones: {str(e)}")
    
    return vis_dir

# Añadir a la parte principal de ejecución
# Preparar datos para entrenamiento y evaluación
def prepare_train_val_data(ds_full, input_window=INPUT_WINDOW, output_horizon=OUTPUT_HORIZON):
    """
    Prepara los datos de entrenamiento y validación desde el dataset completo.
    
    Args:
        ds_full: Dataset completo con variables
        input_window: Tamaño de la ventana de entrada
        output_horizon: Horizonte de predicción
    
    Returns:
        X_train, y_train, X_val, y_val: Arrays de entrenamiento y validación
    """
    print("Preparando datos para entrenamiento y validación...")
    
    # Extraer precipitación como variable objetivo
    # Check if 'precipitacion' exists in the dataset
    if 'precipitacion' in ds_full.data_vars:
        precip = ds_full.precipitacion.values
        print("Using 'precipitacion' as target variable")
    # If not, check for 'total_precipitation'
    elif 'total_precipitation' in ds_full.data_vars:
        precip = ds_full.total_precipitation.values
        print("Using 'total_precipitation' as target variable")
    else:
        # If neither exists, raise an error
        raise ValueError("Could not find precipitation variable in dataset. Available variables: " + 
                        str(list(ds_full.data_vars.keys())))
    
    # Ensure precip has 3 dimensions (time, lat, lon)
    if len(precip.shape) != 3:
        raise ValueError(f"Expected precipitation data to have 3 dimensions (time, lat, lon), got {precip.shape}")
    
    n_times, height, width = precip.shape
    print(f"Precipitation data shape: {precip.shape}")
    
    # Extraer features, por ejemplo cluster y elevación
    features = []
    
    # Verificar si las features estáticas existen y expandirlas para todas las timesteps
    if 'cluster' in ds_full.data_vars:
        cluster_data = ds_full.cluster.values
        # Si cluster es 2D (lat, lon), expandir a 3D (time, lat, lon)
        if len(cluster_data.shape) == 2:
            cluster_data = np.repeat(cluster_data[np.newaxis, :, :], n_times, axis=0)
        features.append(cluster_data)
        print("Añadida variable 'cluster', shape:", cluster_data.shape)
    
    if 'elevation' in ds_full.data_vars:
        elev_data = ds_full.elevation.values
        # Si elevation es 2D (lat, lon), expandir a 3D (time, lat, lon)
        if len(elev_data.shape) == 2:
            elev_data = np.repeat(elev_data[np.newaxis, :, :], n_times, axis=0)
        features.append(elev_data)
        print("Añadida variable 'elevation', shape:", elev_data.shape)
    
    # Si no hay features específicas, usar precipitación histórica como feature
    if not features:
        print("No se encontraron features específicas, usando precipitación histórica")
        # Usar precipitación como característica, añadiendo una dimensión de canal
        features_array = precip.reshape(n_times, 1, height, width)
        print(f"Shape de features: {features_array.shape}")
    else:
        # Concatenar features en un solo array a lo largo de una nueva dimensión (canal)
        features_array = np.stack(features, axis=1)
        print(f"Shape de features combinadas: {features_array.shape}")
    
    # Crear ventanas deslizantes de manera segura
    X, y = [], []
    
    for i in range(n_times - input_window - output_horizon + 1):
        # Input: ventana de datos
        X.append(features_array[i:i+input_window].copy())
        # Output: horizonte de predicción 
        y.append(precip[i+input_window:i+input_window+output_horizon].copy())
    
    # Verificar formas antes de convertir
    if not X or not y:
        raise ValueError("No se pudieron crear ventanas válidas. Verifique los datos de entrada.")
        
    # Usar np.stack en lugar de np.array para garantizar arrays homogéneos
    X = np.stack(X)
    y = np.stack(y)
    
    print(f"Datos preparados - X shape: {X.shape}, y shape: {y.shape}")
    
    # División train/val (80/20)
    split_idx = int(0.8 * len(X))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    print(f"Train shapes - X: {X_train.shape}, y: {y_train.shape}")
    print(f"Val shapes - X: {X_val.shape}, y: {y_val.shape}")
    
    return X_train, y_train, X_val, y_val

# Función para entrenar modelos híbridos
def train_hybrid_model(name, model, train_loader, val_loader, epochs=100, patience=20, 
                      optimizer=None, criterion=None, scheduler=None):
    """
    Entrena un modelo híbrido con optimizaciones avanzadas y realiza evaluación
    """
    print(f"\n{'='*30}")
    print(f"ENTRENAMIENTO DE {name}")
    print(f"{'='*30}")
    
    device = next(model.parameters()).device
    
    # Importar tqdm si no está disponible
    try:
        from tqdm import tqdm
    except ImportError:
        # Definir una versión simple si no está instalado
        def tqdm(iterable, **kwargs):
            print(kwargs.get('desc', ''))
            return iterable
    
    # Optimizador y función de pérdida si no fueron proporcionados
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        print("Usando optimizador Adam por defecto con lr=1e-3")
    
    if criterion is None:
        criterion = nn.MSELoss()
        print("Usando criterio MSELoss por defecto")
    
    if scheduler is None:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=10, verbose=True
        )
        print("Usando scheduler ReduceLROnPlateau por defecto")
    
    # Tracking de métricas
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    
    # Obtener información de las dimensiones de los datos
    sample_batch = next(iter(train_loader))
    sample_inputs, sample_targets = sample_batch
    input_seq_len = sample_inputs.shape[1] if len(sample_inputs.shape) >= 2 else 1
    target_seq_len = sample_targets.shape[1] if len(sample_targets.shape) >= 2 else 1
    print(f"INPUT SEQ LEN: {input_seq_len}, TARGET SEQ LEN: {target_seq_len}")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        batch_losses = []
        
        for batch_data in tqdm(train_loader, desc=f"Época {epoch+1}/{epochs} [Train]", leave=False):
            inputs, targets = batch_data
            
            # Debugging para el primer batch del epoch inicial
            if epoch == 0 and len(batch_losses) == 0:
                print(f"Train batch shapes - inputs: {inputs.shape}, targets: {targets.shape}")
            
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            if epoch == 0 and len(batch_losses) == 0:
                print(f"Train outputs shape: {outputs.shape}")
            
            # Ajustar dimensiones si es necesario
            if len(outputs.shape) == 5 and outputs.shape[2] == 1:  # [batch, seq, 1, H, W]
                outputs = outputs.squeeze(2)  # [batch, seq, H, W]
            
            # NUEVO: Manejar caso especial de targets con 5 dimensiones (doble seq) [batch, seq, seq, H, W]
            if len(targets.shape) == 5 and len(outputs.shape) == 4 and outputs.shape[1] == targets.shape[1]:
                # Seleccionar solo uno de los seq_len (por ejemplo, primera dimensión temporal para cada secuencia)
                targets = targets[:, :, 0, :, :]  # Convertir [batch, seq, seq, H, W] a [batch, seq, H, W]
                print(f"Convertido targets de forma {targets.shape} eliminando dimensión redundante")

            # NUEVO: Manejar caso donde targets tiene forma [batch, seq, seq_target, H, W]
            elif len(targets.shape) == 5 and len(outputs.shape) == 4:
                # Seleccionar diagonal o promediar sobre la 3ra dimensión
                targets_reshaped = []
                for i in range(min(targets.shape[1], outputs.shape[1])):
                    # Para cada paso temporal, quedarse con el primer elemento o con la diagonal
                    if i < targets.shape[2]:
                        targets_reshaped.append(targets[:, i, i])
                    else:
                        targets_reshaped.append(targets[:, i, -1])
                
                # Apilar para formar [batch, seq, H, W]
                targets = torch.stack(targets_reshaped, dim=1)
                print(f"Convertido targets mediante selección diagonal: {targets.shape}")
            
            # NUEVO: Ajuste de secuencia cuando output_seq_len != target_seq_len
            if len(outputs.shape) == 4 and len(targets.shape) == 4:
                output_seq_len = outputs.shape[1]
                target_seq_len = targets.shape[1]
                
                if output_seq_len > target_seq_len:
                    # Si el modelo produce una secuencia más larga, usar solo los primeros target_seq_len elementos
                    outputs = outputs[:, :target_seq_len]
                    print(f"Recortando outputs de {output_seq_len} a {target_seq_len}")
                elif output_seq_len < target_seq_len:
                    # Si el modelo produce una secuencia más corta, repetir el último elemento
                    padding = targets[:, output_seq_len:target_seq_len].shape[1]
                    last_output = outputs[:, -1:].repeat(1, padding, 1, 1)
                    outputs = torch.cat([outputs, last_output], dim=1)
                    print(f"Expandiendo outputs de {output_seq_len} a {target_seq_len}")
            
            # Manejar diferentes formatos de target
            if len(targets.shape) == 3 and len(outputs.shape) == 4:  # targets: [batch, H, W], outputs: [batch, seq, H, W]
                targets = targets.unsqueeze(1).repeat(1, outputs.shape[1], 1, 1)
                
            elif len(targets.shape) == 4 and len(outputs.shape) == 4 and targets.shape[1] == 1:
                # Si targets tiene solo 1 paso temporal pero outputs tiene múltiples
                targets = targets.repeat(1, outputs.shape[1], 1, 1)
            
            # Asegurar dimensiones compatibles antes de calcular la pérdida
            if outputs.shape != targets.shape:
                print(f"⚠️ Advertencia: Las formas siguen siendo incompatibles después de ajustes:")
                print(f"   outputs.shape: {outputs.shape}, targets.shape: {targets.shape}")
                
                # NUEVO: Mejor manejo de dimensiones inconsistentes
                # Caso específico: [batch, seq, H, W] vs [batch, seq, seq, H, W]
                if len(targets.shape) == 5 and len(outputs.shape) == 4:
                    # Colapsar los dos dims de secuencia en targets
                    B, S1, S2, H, W = targets.shape
                    targets = targets.view(B, S1*S2, H, W)
                    # Luego tomar solo los primeros S1 elementos
                    targets = targets[:, :outputs.shape[1]]
                    print(f"   Redimensionado targets a: {targets.shape}")
                
                # Caso genérico: ajustar el mínimo de dimensiones compatibles
                if outputs.shape != targets.shape:
                    try:
                        # Intento de broadcast
                        min_shape = [min(o, t) for o, t in zip(outputs.shape, targets.shape) 
                                   if o is not None and t is not None]
                        slices_out = tuple(slice(0, m) for m in min_shape)
                        slices_tgt = tuple(slice(0, m) for m in min_shape)
                        
                        outputs = outputs[slices_out]
                        targets = targets[slices_tgt]
                        print(f"   Redimensionado a shape común: {outputs.shape}")
                    except Exception as e:
                        print(f"Error en redimensionado: {str(e)}")
                        # Último recurso: aplanar todo y recortar
                        flat_out = outputs.flatten()[:10000]  # Límite para evitar OOM
                        flat_tgt = targets.flatten()[:10000]
                        outputs = flat_out.unsqueeze(0)
                        targets = flat_tgt.unsqueeze(0)
                        print(f"   ⚡ Aplanado a: {outputs.shape}")
            
            # Debugging final shapes
            if epoch == 0 and len(batch_losses) == 0:
                print(f"Adjusted shapes - outputs: {outputs.shape}, targets: {targets.shape}")
            
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            
            batch_losses.append(loss.item())
        
        # Calculate average training loss
        train_loss = sum(batch_losses) / len(batch_losses)
        train_losses.append(train_loss)
        
        # Validation phase
        model.eval()
        val_batch_losses = []
        
        with torch.no_grad():
            for batch_data in tqdm(val_loader, desc=f"Época {epoch+1}/{epochs} [Val]", leave=False):
                inputs, targets = batch_data
                
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                # Forward pass
                outputs = model(inputs)
                
                # Aplicar los mismos ajustes que en el entrenamiento
                if len(outputs.shape) == 5 and outputs.shape[2] == 1:
                    outputs = outputs.squeeze(2)
                    
                # Manejar caso especial de targets con 5 dimensiones
                if len(targets.shape) == 5 and len(outputs.shape) == 4 and outputs.shape[1] == targets.shape[1]:
                    targets = targets[:, :, 0, :, :]

                # Manejar dimensión de secuencia extra en targets
                elif len(targets.shape) == 5 and len(outputs.shape) == 4:
                    targets_reshaped = []
                    for i in range(min(targets.shape[1], outputs.shape[1])):
                        if i < targets.shape[2]:
                            targets_reshaped.append(targets[:, i, i])
                        else:
                            targets_reshaped.append(targets[:, i, -1])
                    targets = torch.stack(targets_reshaped, dim=1)
                
                # Ajuste de longitud de secuencia
                if len(outputs.shape) == 4 and len(targets.shape) == 4:
                    output_seq_len = outputs.shape[1]
                    target_seq_len = targets.shape[1]
                    
                    if output_seq_len > target_seq_len:
                        outputs = outputs[:, :target_seq_len]
                    elif output_seq_len < target_seq_len:
                        padding = targets[:, output_seq_len:target_seq_len].shape[1]
                        last_output = outputs[:, -1:].repeat(1, padding, 1, 1)
                        outputs = torch.cat([outputs, last_output], dim=1)
                
                if len(targets.shape) == 3 and len(outputs.shape) == 4:
                    targets = targets.unsqueeze(1).repeat(1, outputs.shape[1], 1, 1)
                    
                elif len(targets.shape) == 4 and len(outputs.shape) == 4 and targets.shape[1] == 1:
                    targets = targets.repeat(1, outputs.shape[1], 1, 1)
                
                # Compatibilidad final de formas
                if outputs.shape != targets.shape:
                    # Caso específico: [batch, seq, H, W] vs [batch, seq, seq, H, W]
                    if len(targets.shape) == 5 and len(outputs.shape) == 4:
                        B, S1, S2, H, W = targets.shape
                        targets = targets.view(B, S1*S2, H, W)[:, :outputs.shape[1]]
                    
                    # Caso genérico
                    if outputs.shape != targets.shape:
                        try:
                            min_shape = [min(o, t) for o, t in zip(outputs.shape, targets.shape) 
                                       if o is not None and t is not None]
                            slices_out = tuple(slice(0, m) for m in min_shape)
                            slices_tgt = tuple(slice(0, m) for m in min_shape)
                            
                            outputs = outputs[slices_out]
                            targets = targets[slices_tgt]
                        except:
                            # Último recurso
                            flat_out = outputs.flatten()[:10000]
                            flat_tgt = targets.flatten()[:10000]
                            outputs = flat_out.unsqueeze(0)
                            targets = flat_tgt.unsqueeze(0)
                
                loss = criterion(outputs, targets)
                val_batch_losses.append(loss.item())
        
        # Calculate average validation loss
        val_loss = sum(val_batch_losses) / len(val_batch_losses)
        val_losses.append(val_loss)
        
        # Update learning rate if scheduler is provided
        if scheduler is not None:
            scheduler.step(val_loss)
        
        # Print progress
        if epoch % 5 == 0 or epoch == epochs - 1:
            print(f"{name} - Epoch {epoch+1}/{epochs}: Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            # Save the best model state
            best_model_state = {k: v.clone() for k, v in model.state_dict().items()}
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs!")
                break
    
    # Restore best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    print(f"Training completed for {name}. Best validation loss: {best_val_loss:.6f}")
    return model, train_losses, val_losses

# Preparar datos para entrenamiento
X_train, y_train, X_val, y_val = prepare_train_val_data(ds_full)

# Entrenar modelos SimpleConvGRU y SimpleConvLSTM y comparar
print("\nEntrenando modelos simples y comparando con híbridos...")

# Entrenar y comparar modelos con los datos preparados
comparison_models = train_and_compare_models(X_train, y_train, X_val, y_val)
visualize_model_comparisons(comparison_models)

# Mostrar mensaje de éxito
print("\n✅ Comparación completa. Todos los resultados guardados en " + str(MODELS_OUTPUT / 'visualization' / 'comparisons'))